In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[124]:


#Importing necessary libraries
from bs4 import BeautifulSoup as bs
import html5lib
import requests as rq
import pandas as pd
import numpy as np
import json


# In[127]:


#Scrapping using BeautifulSoup and converting t
url = 'https://www.cermati.com/karir'
HEAD = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
webpage = rq.get(url, headers=HEAD)
soup = bs(webpage.content, "html.parser")
print("The Webpage responded with", webpage)


# In[4]:


#Extracting the required data and converting to dictionary
ext = soup.find("script", attrs={'id': "initials" }).text
jobs_list = json.loads(ext)['smartRecruiterResult']['all']['content']


# In[123]:


#Extracting the Job ID's
job_ids={}
for i in jobs_list:
    job_ids[i['name']]=i['id']
print("Total jobs found is", len(job_ids))


# In[ ]:


"""
1. We can get the department, location, job_type details from main url page
2. we can get the description and qualification from job url page
So, We are going to get the data from both page and merge to form a json file
"""


# In[81]:


# 2.1 Defining a function to fetch job details from each job page
def full_jd(o):
    url = 'https://www.smartrecruiters.com/Cermaticom/' + o
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }
    webpage = rq.get(url, headers=headers)
    soup = bs(webpage.content, 'html.parser')
    fin_jd = {}
    try:
        job_desc = soup.find("div", attrs={'class': 'wysiwyg', 'itemprop':'responsibilities'}).find("ul").find_all('li')
        a1 = []
        for i in job_desc:
            a1.append(i.text)
        job_qual = soup.find("div", attrs={'class': 'wysiwyg', 'itemprop':'qualifications'}).find("ul").find_all('li')
        a2 = []
        for i in job_qual:
            a2.append(i.text)
            fin_jd = {'Description': a1, 'Qualification': a2}
        return fin_jd
    except:
        pass


# In[82]:


# 2.2 Forming a combined description and qualification jd of all jobs
print("Fetching the Job details.....")
big_jd = {}
for i, j in job_ids.items():
    big_jd[i] = full_jd(j)


# In[94]:


# 2.3 Defining a function to remove NOne values
def remove_none_values(dictionary):
    new_dictionary = {}
    try:
        for key, value in dictionary.items():
            if value is not None:
                new_dictionary[key] = value
        return new_dictionary
    except:
        pass
big_jd = remove_none_values(big_jd)


# In[69]:


# 1.1 Defining functions to fetch data from main url page
def find_dep(o):
    try:
        return o['department']['label']
    except:
        pass
def jd(l):
    n = {}
    try:
        n['title'] = l['name']
        n['location'] = l['location']['city'] + ", " + l['customField'][2]['valueLabel']
        n['release_date'] = l['releasedDate']
        n['job_type'] = l['typeOfEmployment']['id'] + ", " + l['typeOfEmployment']['label']
        return n
    except:
        pass


# In[20]:


# 1.2 Creating a for loop to iterate over each job in URL main page
print("Fetching small JD")
small_jd = {}
for s in jobs_list:
    if find_dep(s) in small_jd.keys():
        small_jd[find_dep(s)].append(jd(s))
    else:
        lister = []
        lister.append(jd(s))
        small_jd[find_dep(s)] = lister


# In[22]:


# 1.3 Removing None items
small_jd.pop(None)


# In[101]:


# Merging output small_jd and big_jd
def merging_jd(small_jd, big_jd):
    for i, j in small_jd.items():
        for o in small_jd[i]:
            for l, m in big_jd.items():
                if o['title'] == l:
                    o.update(big_jd[l])
    return small_jd


# In[102]:


# Merging function
solution = merging_jd(small_jd, big_jd)


# In[106]:


# Converting to json
solution_json = json.dumps(solution, indent=4)


# In[ ]:


with open('solution.json', mode='wt', encoding='utf-8') as saver:
    saver.writelines(solution_json)
print("The Final json is stored in local directory")

The Webpage responded with <Response [200]>
Total jobs found is 96
Fetching the Job details.....
Fetching small JD
The Final json is stored in local directory
